In [1]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from sklearn.model_selection import train_test_split
from huggingface_hub import notebook_login
import torch


In [2]:
notebook_login()


In [ ]:
dataset = load_dataset("SKNahin/bengali-transliteration-data")

dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

In [6]:
dataset = DatasetDict({
    "train": dataset["train"],
    "validation": dataset["test"]
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 4004
    })
    validation: Dataset({
        features: ['bn', 'rm'],
        num_rows: 1002
    })
})


In [11]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [ ]:
model_name='facebook/mbart-large-50-many-to-many-mmt'
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
#this process took huge time as it is very big compared to the internet bandwith, and it is still running so the next steps could not be run until its finishing


NameError: name 'MBartForConditionalGeneration' is not defined

In [15]:
def preprocess_function(examples):
    inputs = examples["banglish"]
    targets = examples["bangla"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_strategy="epoch",
    save_strategy="epoch"
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
#training start using the provided dataset for fine tuning
trainer.train()

In [ ]:
model.save_pretrained("banglish-to-bangla-model")
tokenizer.save_pretrained("banglish-to-bangla-model")